In [ ]:
!python -m pip install --upgrade pip

In [ ]:
!pip install typing_extensions pydantic openai

In [ ]:
!pip install datasets transformers peft trl bitsandbytes

In [ ]:
from huggingface_hub import login
login(token='')

In [ ]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model, PeftModel
import bitsandbytes as bnb
import torch.nn.functional as F

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
# base 모델 담는 객체
model_name = "google/gemma-3-4b-it"

In [ ]:
bnb_config = {
    "load_in_4bit": True,
    "bnb_4bit_compute_dtype": torch.float16,
    "bnb_4bit_quant_type": "nf4",
    "device_map": "auto"
}

In [ ]:
# 토크나이저 및 모델 로드 (모델 로드 시 4-bit 양자화 설정)
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(model_name, **bnb_config)
# base model 작업 끝

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

---
### 학습용 데이터 로드

In [ ]:
# 데이터 전처리 함수
def preprocess_data(examples):

    # 입력 데이터 (질문 + 문맥)
    inputs = [q,'답변:',a for q,a in zip(examples["question"], examples["answers"])]
    # 정답 데이터 추출
    answer_texts = [for a in examples["answers"]]

    # 입력 데이터 토큰화
    model_inputs = tokenizer(
        inputs,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    )

    # 정답 데이터 토큰화
    labels = tokenizer(
        answer_texts,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    )["input_ids"]

    # input_ids 기준으로 labels 길이 맞춤
    max_length = model_inputs["input_ids"].shape[1]
    labels = labels[:, :max_length]

    # 패딩된 부분을 -100으로 설정 (loss 계산에서 무시)
    labels[labels == tokenizer.pad_token_id] = -100

    model_inputs["labels"] = labels

    return model_inputs

In [ ]:
# 데이터셋 전처리 적용
train_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset.column_names)
val_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset.column_names)

In [ ]:
print(tokenizer.decode(train_dataset[0]["input_ids"], skip_special_tokens=True))

---

In [ ]:
# LoRA 설정
lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA 적용
model = get_peft_model(base_model, lora_config)
model.enable_input_require_grads()
model.gradient_checkpointing_enable()
model.print_trainable_parameters()

In [ ]:
training_args = TrainingArguments(
    output_dir='./q_lora_korqa',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    weight_decay=0.01,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=2,
    fp16=True,
    push_to_hub=False,
    report_to='none'
)

# 주석에 데이터를 추가해야함

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [ ]:
trainer.train()

---

학습후 불러오기

In [ ]:
from transformers import AutoConfig

trained_model_path = "./q_lora_korqa/checkpoint-5661"

config = AutoConfig.from_pretrained(model_name)
config.save_pretrained(trained_model_path)

In [ ]:
adapter_model_path = "./q_lora_korqa/checkpoint-5661"

base_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")
model = PeftModel.from_pretrained(base_model, adapter_model_path)

qa_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

In [ ]:
question = ""
context = ""

input_text = f"질문: {question}\n문맥: {context}\n답변:"

output = qa_pipeline(input_text, max_new_tokens=50, temperature=0.7, top_p=0.8)

print(output)